-------------------------------------------------------------------
# Homework set 7 - solution

 * **Author**: Balint Szoke
 * **UniID**: bs2574

-------------------------------------------------------------------


In [1]:
import numpy as np

## Exercise 1

**Primitives of the problem:**
 * transition probabilities for the states: $q(x,y)$ (also informs us about the cardinality of the sate space $|S|=N$). With $N<\infty$, $q$ can be represented as an $N\times N$ matrix $\mathbf{Q}$
 * dividend function $d$, which is an $N$-vector $\mathbf{d}$ (provided that $N<\infty$) with values from $(0,\infty)$ 
 * discount factor $\beta \in (0,1)$
 
**Informal idea (value function iteration):**
 - assuming a finite $N$, we can take the above primitives as given and express the $T$ operator using matrix notation
 
 $$T\mathbf{p} = \mathbf{Q}[\mathbf{d} + \mathbf{p}]$$
 - then the idea is to start with an arbitrary $\mathbf{p}_0$ (because of the global stability of $T$, it can be anything, say $\mathbf{p}_0=\mathbf{0}_N$) and iterate on the $T$ operator 
 
 $$T^{k+1}\mathbf{p}_0 = T (T^{k}\mathbf{p}_0)$$
 until $T^{k+1}\mathbf{p}_0 \approx T^{k}\mathbf{p}_0$.

In [2]:
def price_function(Q, d, beta, tol = 1e-4):

    N = Q.shape[0]
    
    if N != Q.shape[1]:
        raise ValueError("Q must be a square matrix!")
    
    elif not all(np.isclose(np.sum(Q, 1), np.ones(N))):
        raise ValueError("Q must be a stochastic matrix!")    
    
    elif N != d.shape[0]:
        raise ValueError("The size of the dividend vector doesn't match with the size of the state space")
    
    diff_inf = 1.0
    p0 = np.zeros(N)
    
    while diff_inf > tol:
        p1 = beta * (Q @ (d + p0))
        diff_inf = max(abs(p1 - p0))
        p0 = p1
    
    return p1

Try this function with the provided parameter values

In [3]:
d = np.asarray([1.0, 0.0, 0.0])

beta = .98

Q = np.asarray([[.971, .029, .0  ],
                [.145, .778, .077],
                [.0  , .508, .492]])

Q2 = np.asarray([[.871, .029, .1  ],
                 [.145, .778, .077],
                 [.0  , .508, .492]])


print("Price function for the benchmark q")
print(price_function(Q, d, beta))
print('\n')
print("Price function for the q with a more likely bad state")
print(price_function(Q2, d, beta))

Price function for the benchmark q
[ 40.75491039  35.95601847  34.5671384 ]


Price function for the q with a more likely bad state
[ 23.49629465  21.09399103  20.27911265]


**Interpretation:**

We can see that for all states the price is lower when the probability of the worst state (from which there is no come back to the good state) is higher. The reason for this is that the more pessimistic (second) belief "overweights" the eventually zero dividend sequences, so naturally the implied mean value is lower than in the case of the first $q$. Notice also that this is not a risk-premium story; the agent is risk-neutral so the price effect comes through the differences in the first moments.

## Exercise 2


**Sufficient conditions for a unique fixed point of $\bar{T}$:**    $\quad d \in b\mathbb{R}^{S}$

where the operator $\bar{T}$ is defined as 

$$ \left(\bar{T}p\right)(x) = \beta \ \max\limits_{i \in \{1,2\}} \Big\{ \sum_{y \in S} [d(y) + p(y)] q_i(x,y)\Big\} \quad ( x \in S ) $$
with $q_1$ and $q_2$ being stochastic matrices. We know that $(b\mathbb{R}^S,d_{\infty})$ is a complete Banach space, so all we need is to show is that $\bar{T}$ is a uniform contraction. To this end, 

---------------------------------
*Proof:*

First, check that $\bar{T}$ is a self-map on $(b\mathbb{R}^S,d_{\infty})$, i.e. it maps $p \in b\mathbb{R}^{S}$ into $\bar{T}p \in b\mathbb{R}^{S}$

\begin{align*}
\left|(\bar{T}p)(x) \right| &= \left| \hspace{1mm}\beta \ \max_{i \in \{1,2\}} \Big\{ \sum_{y \in S} [d(y) + p(y)] q_i(x,y)\Big\} \right| \leq \\ &\leq \beta \ \max_{i \in \{1,2\}} \Bigg\{ \Big| \sum_{y \in S} [d(y) + p(y)] q_i(x,y) \Big| \Bigg\} \leq \beta \ \max_{i \in \{1,2\}} \Bigg\{  \sum_{y \in S} \Big| d(y) + p(y) \Big| q_i(x,y)  \Bigg\} \\ &\leq \beta \ \left( || d ||_{\infty} + || p ||_{\infty} \right)
\end{align*}

taking the supremum over $x\in S$ gives us $||\bar{T}p||_{\infty}<\infty$.


To see the uniform contraction property, take $p, p'\in b\mathbb{R}^S$ and use the following inequality (see the proof below)

$$\left|\sup_i a_i - \sup_i b_i \right| \leq \sup_i \left|a_i - b_i\right|$$

to obtain

\begin{align*}
\big|(\bar{T}p)(x) - (\bar{T}p')(x)\big| &= \beta  \ \Bigg| \max_{i \in \{1,2\}} \Bigg\{ \sum_{y \in S} [d(y) + p(y)] q_i(x,y)\Bigg\} - \max_{i \in \{1,2\}} \Bigg\{ \sum_{y \in S} [d(y) + p'(y)] q_i(x,y)\Bigg\} \Bigg| \\ &\leq  \beta  \max_{i \in \{1,2\}} \Bigg\{ \Big| \sum_{y \in S} [p(y) - p'(y)] q_i(x,y) \Big| \Bigg\}  \\ &\leq \beta \ || p - p'||_{\infty}
\end{align*}

taking the supremum over $x\in S$ gives us $||\bar{T}p - \bar{T}p'||_{\infty}\leq \beta ||p - p'||_{\infty}$ with $\beta\in(0,1)$, so $\bar{T}$ is indeed a uniform contraction if $d \in b\mathbb{R}^{S}$

--------------------------------------------
**CLAIM:** For any $f,g \in b\mathbb{R}^S$, 

$$\left|\sup_{x\in S} f(x) - \sup_{x\in S} g(x) \right| \leq \sup_{x\in S} \left|f(x) - g(x)\right|$$

--------------------------------------------
*Proof:*

Because $f = f - g + g \leq \left|f - g\right| + g$, it follows that

$$\sup_{x\in S} f(x) \leq \sup_{x\in S} \left|f(x)-g(x)\right| + \sup_{x\in S} g(x)\quad \Rightarrow \quad \sup_{x\in S} f(x) - \sup_{x\in S} g(x) \leq \sup_{x\in S} \left|f(x)-g(x)\right|$$

Changing the role of $f$ and $g$ in the above argument implies 

$$\sup_{x\in S} g(x) - \sup_{x\in S} f(x) \leq \sup_{x\in S} \left|f(x)-g(x)\right|\quad \Rightarrow \quad \sup_{x\in S} f(x) - \sup_{x\in S} g(x) \geq - \sup_{x\in S} \left|f(x)-g(x)\right|$$

## Exercise 3

In [4]:
def price_function_hetero(Q1, Q2, d, beta, tol = 1e-4):

    N = Q1.shape[0]
    
    if N != Q1.shape[1] or Q2.shape[0] != Q2.shape[1]:
        raise ValueError("Qs must be square matrices!")
    elif Q1.shape != Q2.shape:
        raise ValueError("The two Q matrices must be of the same size!")
    
    elif not all(np.isclose(np.sum(np.vstack([Q1, Q2]), 1), np.ones(2 * N))):
        raise ValueError("Qs must be stochastic matrices!")    
    
    elif N != d.shape[0]:
        raise ValueError("The size of the dividend vector doesn't match with the size of the state space")
    
    diff_inf = 1.0
    p0 = np.zeros(N)
    
    while diff_inf > tol:
        p1 = beta * np.maximum(Q1 @ (d + p0), Q2 @ (d + p0))
        diff_inf = max(abs(p1 - p0))
        p0 = p1
    
    return p1

In [5]:
print("Price function for the heterogeneous beliefs model:")
print(price_function_hetero(Q, Q2, d, beta))

Price function for the heterogeneous beliefs model:
[ 40.75491039  35.95601847  34.5671384 ]
